https://www.tensorflow.org/get_started/get_started
https://www.tensorflow.org/get_started/mnist/beginners

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

Two steps:

1) building a graph

2) running the graph

Computational graph is made of nodes each performing a defined operation.

In [9]:
n1 = tf.constant(3, dtype = tf.float32) # constant node (no input)
n2 = tf.constant(4, dtype = tf.float32)
print(n1)

Tensor("Const_9:0", shape=(), dtype=float32)


In [13]:
sesh = tf.Session() # session for executing the graph
output = sesh.run([n1, n2])
print(output)

[3.0, 4.0]


In [16]:
n3 = tf.add(n1, n2) # node adding two values together
print({'n3' : n3})
output_add = sesh.run(n3)
print(output_add)

{'n3': <tf.Tensor 'Add_1:0' shape=() dtype=float32>}
7.0


In [21]:
a = tf.placeholder(tf.float32) # placeholder represents a parametrized input, providing an exact value later
b = tf.placeholder(tf.float32)
n4 = a + b # equivalent to tf.add(a, b)
print(sesh.run(n4, {a : 2, b : 2}))
print(sesh.run(n4, {a : [2, 2, 6], b : [1, 2, 3]})) # "vectorized" input

4.0
[ 3.  4.  9.]


In [23]:
n5 = (a + b) * 3
print(sesh.run(n5, {a : [2, 2, 6], b : [1, 2, 3]}))

[  9.  12.  27.]


In [37]:
w = tf.Variable([ .3], dtype = tf.float32) # varibles represents trainable parameters of a graph (weights, biases etc...)
b = tf.Variable([-.3], dtype = tf.float32)
x = tf.placeholder(tf.float32) # data input
lm = w * x + b # linear model

# to initiliaze thev ariables, you run:
init = tf.global_variables_initializer()
sesh.run(init)

# to run the model:
print(sesh.run(lm, {x : [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [39]:
y = tf.placeholder(tf.float32) # desired values

# defining loss function (how far the model and y are apart):
squared_deltas = tf.square(lm - y) # squared diffenrences
loss = tf.reduce_sum(squared_deltas) # sum to get a scalar output value

print(sesh.run(loss, {x : [1, 2, 3, 4], y : [0, -1, -2, -3]})) # loss value for y = [0, -1, -2, -3]

23.66


In [46]:
w_new = tf.assign(w, [-1]) # change the parameters
b_new = tf.assign(b, [ 1]) 
print(sesh.run([w_new, b_new]))
print(sesh.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})) # new loss

[array([-1.], dtype=float32), array([ 1.], dtype=float32)]
0.0


In [48]:
# train to optimize the loss function
learning_rate = 0.01
# automatically computes the partial derivatives from the model description
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

sesh.run(init) # to reset variables

for i in range(1000): # train is connected to the loss, the loss is connected to the lam
    sesh.run(train, {x : [1, 2, 3, 4], y : [0, -1, -2, -3]})
    
print(sesh.run([w, b])) # new parameters

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]
